In [5]:
!pip install python-telegram-bot==13.7
!pip install firebase_admin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate('selfcare-database-firebase-adminsdk-bur73-b4e578ff6b.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

In [38]:
from telegram.ext import *
import random
from enum import Enum
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, Update

f = open("telepass.txt", "r")
TOKEN = f.readline()

class Commands(Enum):
    START = "/start"
    HELP = "/help"
    FOOD = "/food"
    FITNESS = "/fitness"
    GOOD_TO_GO = "/random"
  
class Channels(Enum):
    FOOD = -1001808302133
    FITNESS = -1001501471243
    GOOD_TO_GO = -1001578261713

posts = {
    "FOOD" : [7, 2, 3],
    "FITNESS" : [8, 2, 3],
    "GOOD_TO_GO" : [2]
}

print('Starting up bot...')

# Lets us use the /start command
def start_command(update, context):
    reply_keyboard = [["/send", "/remind", "/food", "/fitness", "/random"]]

    update.message.reply_text("Hello. Welcome to wellness telebot",
        reply_markup=ReplyKeyboardMarkup(
            reply_keyboard, one_time_keyboard=True, input_field_placeholder="Try a command!"
        ),
    )

def send_command(update, context):
    update.message.reply_text('Send me image of your best moments!')
    return 0

def photo_command(update, context):
    doc_ref = db.collection(u'users').document(str(update.message.chat.id))

    if doc_ref.get().exists:
        doc_ref.update({u'message_id': firestore.ArrayUnion([update.message.message_id])})
    else:
        Obj = {
            'message_id' : [update.message.message_id],
        }
        doc_ref.set(Obj) 

    update.message.reply_text('Success! Photo received!')
    return ConversationHandler.END

def cancel_command(update, context):
    update.message.reply_text('OK cancel last command!')
    return ConversationHandler.END

def remind_command(update, context):
    update.message.reply_text('Here is an image of your best memories!')
    doc_ref = db.collection(u'users').document(str(update.message.chat.id))
    doc = doc_ref.get()
    if doc.exists:
        chat_id = update.message.chat.id
        message_id = random.choice(doc.to_dict()['message_id'])
        context.bot.forward_message(chat_id, chat_id, message_id)
    else:
        update.message.reply_text('You have not shared any good memories with me yet! Try sending some with /send')

def activity_command(update, context):
    dest_id = update.message.chat.id
    from_chat_id = None
    postList = None
    if update.message.text == Commands.FOOD.value:
        from_chat_id = Channels.FOOD.value
        postList = posts.get("FOOD")
    elif update.message.text == Commands.FITNESS.value:
        from_chat_id = Channels.FITNESS.value
        postList = posts.get("FITNESS")
    elif update.message.text == Commands.GOOD_TO_GO.value:
        from_chat_id = Channels.GOOD_TO_GO.value
        postList = posts.get("GOOD_TO_GO")

    if from_chat_id == None or postList == None:
        update.message.reply_text('error')
    else:
        message_id = postList[0]
        if len(postList) > 1:
            postList.pop(0)
        context.bot.forward_message(dest_id, from_chat_id, message_id)

def activities_handler(update, context):
    l = None
    if update.message.chat.id == Channels.FOOD.value:
        l = posts["FOOD"]
    elif update.message.chat.id == Channels.FITNESS.value:
        l = posts["FITNESS"]
    elif update.message.chat.id == Channels.GOOD_TO_GO.value:
        l = posts["GOOD_TO_GO"]
    if l != None:
        l.append(update.message.message_id)
        # keep recent 5 messages :)
        while len(l) > 5:
            l.pop(0)

def handle_message(update, context):
    message_type = update.message.chat.type
    text = str(update.message.text)
    print(f'User ({update.message.chat.id}) says: "{text}" in: {message_type}')
    update.message.reply_text("Sorry I do not understand. Please enter a valid command!")

def main():
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher

    # Commands
    dp.add_handler(CommandHandler('start', start_command))
    dp.add_handler(CommandHandler('remind', remind_command))
    dp.add_handler(CommandHandler('food', activity_command))
    dp.add_handler(CommandHandler('fitness', activity_command))
    dp.add_handler(CommandHandler('random', activity_command))

    dp.add_handler(MessageHandler(Filters.chat_type.channel, activities_handler))
    
    # converstions
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler("send", send_command)],
        states={
            0 : [MessageHandler(Filters.photo, photo_command)],
        },
        fallbacks=[CommandHandler("cancel", cancel_command)],
    )
    dp.add_handler(conv_handler)

    # Messages
    dp.add_handler(MessageHandler(Filters.text, handle_message))

    # Run the bot
    updater.start_polling(1.0)
    updater.idle()

# Run the program
if __name__ == '__main__':
    main()

Starting up bot...


Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/hojunhao/opt/anaconda3/lib/python3.9/site-packages/telegram/ext/updater.py", line 646, in _network_loop_retry
    if not action_cb():
  File "/Users/hojunhao/opt/anaconda3/lib/python3.9/site-packages/telegram/ext/updater.py", line 597, in polling_action_cb
    updates = self.bot.get_updates(
  File "/Users/hojunhao/opt/anaconda3/lib/python3.9/site-packages/telegram/ext/extbot.py", line 222, in get_updates
    updates = super().get_updates(
  File "/Users/hojunhao/opt/anaconda3/lib/python3.9/site-packages/telegram/bot.py", line 130, in decorator
    result = func(*args, **kwargs)
  File "/Users/hojunhao/opt/anaconda3/lib/python3.9/site-packages/telegram/bot.py", line 2861, in get_updates
    self._post(
  File "/Users/hojunhao/opt/anaconda3/lib/python3.9/site-p